In [1]:
from glob import glob
import os

import cv2
import numpy as np
from PIL import Image
from pdf2image import convert_from_path
from imutils import resize
from tqdm import tqdm

good = list(glob("../data/original/Good/*.pdf"))
ugly = list(glob("../data/original/Ugly/*.pdf"))

len(good), len(ugly)

(16, 25)

In [2]:
def im_resize(img, size=1500):
    height, width = img.shape[:2]

    if height * width > 59478485:
        scale_percent = 50
        new_width = int(width * scale_percent / 100)
        new_height = int(height * scale_percent / 100)
        img = cv2.resize(img, (new_width, new_height), interpolation=cv2.INTER_AREA)

    longer = np.argmax(img.shape)

    if longer == 0:
        img = resize(img, size)
    else:
        img = resize(img, height=size)

    return img

In [3]:
def detect_objects(img):
    gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)

    thresh = cv2.adaptiveThreshold(
        gray, 255,
        cv2.ADAPTIVE_THRESH_GAUSSIAN_C,
        cv2.THRESH_BINARY_INV,
        11, 2
    )

    kernel = np.ones((3,3), np.uint8)
    dilated = cv2.dilate(thresh, kernel, iterations=1)

    contours, hierarchy = cv2.findContours(
        dilated,
        cv2.RETR_CCOMP,
        cv2.CHAIN_APPROX_SIMPLE
    )

    areas = []

    for cnt in contours:
        areas.append(cv2.contourArea(cnt))

    def get_sorted_indices(data):
        return [i[0] for i in sorted(enumerate(data), key=lambda x: x[1], reverse=True)]

    sorted_indices = get_sorted_indices(areas)

    indices = np.expand_dims(np.arange(hierarchy[0].shape[0]), 1)
    hierarchy_area = np.concatenate((indices, hierarchy[0], np.expand_dims(areas, 1)), axis=1)

    return sorted_indices, contours, hierarchy_area

In [4]:
def calculate_border_score(img, contours, sorted_indices):
    width, height = img.shape[:2]
    image_area = width * height
    area_1 = cv2.contourArea(contours[sorted_indices[0]])
    area_2 = cv2.contourArea(contours[sorted_indices[1]])

    return 1 - ((((image_area - area_1) / image_area) + ((area_1 - area_2) / area_1)) / 2)


In [5]:
def process(files, output_dir):
    os.makedirs(output_dir, exist_ok=True)
    font = cv2.FONT_HERSHEY_SIMPLEX
    fontScale = 1
    color = (255, 0, 0)
    thickness = 2

    progress = tqdm(total=len(good))

    for f in files:
        pages = convert_from_path(f, dpi=300)
        img = np.array(pages[0])
        img = im_resize(img)
        img_comp = img.copy()

        sorted_indices, contours, hierarchy_area = detect_objects(img)

        index = sorted_indices[0]
        x, y, w, h = cv2.boundingRect(contours[index])
        cv2.rectangle(img_comp, (x, y), (x+w, y+h), (0, 255, 0), 10)

        index = sorted_indices[1]
        x, y, w, h = cv2.boundingRect(contours[index])
        cv2.rectangle(img_comp, (x, y), (x+w, y+h), (0, 255, 0), 10)

        border_score = calculate_border_score(img, contours, sorted_indices)
        cv2.putText(img_comp, f'{border_score}', (x, y), font, fontScale, color, thickness, cv2.LINE_AA)

        cv2.imwrite(f"{output_dir}/{os.path.basename(f)}.{border_score:.2f}.jpg", img_comp)
        progress.update(1)

In [6]:
process(ugly, "../data/output/ugly")

 44%|████▍     | 7/16 [00:03<00:05,  1.78it/s]/home/sadid/anaconda3/lib/python3.12/site-packages/PIL/Image.py:3186: DecompressionBombWarning: Image size (139513096 pixels) exceeds limit of 89478485 pixels, could be decompression bomb DOS attack.
  warnings.warn(
24it [00:12,  2.33it/s]                        /home/sadid/anaconda3/lib/python3.12/site-packages/PIL/Image.py:3186: DecompressionBombWarning: Image size (139513096 pixels) exceeds limit of 89478485 pixels, could be decompression bomb DOS attack.
  warnings.warn(
25it [00:13,  1.81it/s]


In [7]:
process(good, "../data/output/good")

100%|██████████| 16/16 [00:07<00:00,  2.01it/s]
